# Calculating Charity Scores from relevant charity data on charitydata.ca (using dataframe from Data_Collection.ipyntb)

In [1]:
#importing required libraries and setting the driver path
import pandas as pd
import numpy as np
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

driver_path = "C:/Users/16475/Documents/GitHub/CharityScore.ca/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)

In [2]:
# import the url data from the dataframe
df = pd.read_csv("expense data.csv")

# remove duplicate index column
df = pd.read_csv("expense data.csv").drop(['Unnamed: 0'],axis=1)

print(df.head(1))

                                                 URL       Name BRN  \
0  https://www.charitydata.ca/charity/full-name/0...  full name   1   

  Expense Table Expense_Table  
0           NaN           NaN  


In [3]:
# find expense tables on charitydata.ca and add them to the dataframe as string representations of dictionaries

for i in range(1, len(df)): 
    driver.get(df.loc[i,"URL"])
    
    # check if url page found, else skip
    try:
        url_check = driver.find_element_by_xpath("/html/body/main/div/section[2]/div/div[2]/p").text
    except NoSuchElementException:
        url_check = ""
    
    if url_check == "The requested page could not be found.":
        continue
    
    # open expense table
    try:
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.LINK_TEXT, 'Expenses'))).click()
        expense_table = driver.find_element_by_xpath('//html/body/main/div/section[3]/div/div[2]/div[4]/div[2]/table[1]')
    except TimeoutException:
        try:
            driver.get(df.loc[i,"URL"])
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.LINK_TEXT, 'Expenses'))).click()
            expense_table = driver.find_element_by_xpath('//html/body/main/div/section[3]/div/div[2]/div[4]/div[2]/table[1]')
        except:
            continue
    
    # create dictionary representation of expense table
    expense_dict = {}
    
    expense_rows = expense_table.find_elements(By.TAG_NAME, "tr")
    header_row = expense_rows.pop(0)
    
    headers = header_row.find_elements(By.TAG_NAME, "th")
    header_key = headers.pop(0).text
    headers = list(map(lambda x: int(x.text), headers))
    expense_dict[header_key] = headers 
    
    for row in expense_rows:
        row_key = row.find_element(By.TAG_NAME, "th").text
        row_series = row.find_elements(By.TAG_NAME, "td")
        num_list = []
        
        for num in row_series:
            if num.text[0] != "(":
                num_list +=[int(num.text.replace(",",""))]
            else:
                num_list += [int(num.text[1:-1].replace(",",""))]
                
        expense_dict[row_key] = num_list
        
    df.loc[df.index[i], 'Expense_Table'] = str(expense_dict)
    df.to_csv("expense data.csv")
    
# quit driver and export to csv  
driver.quit()
df.to_csv("expense data.csv")
    
    
    
    

KeyError: 84079